In [2]:
import os
from langchain_openai import OpenAI
from langchain import SQLDatabase
from snowflake.snowpark import Session
from langchain.chains import create_sql_query_chain

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
password = os.environ.get("SNOW_PASS")
snowflake_account = os.environ.get("SNOW_ACCOUNT")
role = "ENGINEER"
username = os.environ.get("SNOW_USER")
database = "DEADPOOL"
schema = "PROD"
warehouse = "ENGINEER"

In [4]:
snowflake_url = f"snowflake://{username}:{password}@{snowflake_account}/{database}/{schema}?warehouse={warehouse}&role={role}"

db = SQLDatabase.from_uri(snowflake_url,
                          sample_rows_in_table_info=1, 
                          include_tables=['picks','players'])

# we can see what information is passed to the LLM regarding the database
print(db.table_info)


CREATE TABLE picks (
	name VARCHAR(256), 
	birth_date DATE, 
	death_date DATE, 
	age DECIMAL(38, 0), 
	picked_by VARCHAR(256), 
	wiki_page VARCHAR(256), 
	year DECIMAL(38, 0), 
	timestamp TIMESTAMP_NTZ, 
	wiki_id VARCHAR(256)
)

/*
1 rows from picks table:
name	birth_date	death_date	age	picked_by	wiki_page	year	timestamp	wiki_id
Henry Kissinger	1923-05-27	2023-11-29	100	1831699b-e255-45ff-8671-b5c840922735	Henry_Kissinger	2023	None	Q66107
*/


CREATE TABLE players (
	first_name VARCHAR(256), 
	last_name VARCHAR(256), 
	email VARCHAR(256), 
	year_one DECIMAL(38, 0), 
	year_two DECIMAL(38, 0), 
	opt_in BOOLEAN, 
	sms VARCHAR(256), 
	id VARCHAR(36), 
	password VARCHAR(256)
)

/*
1 rows from players table:
first_name	last_name	email	year_one	year_two	opt_in	sms	id	password
Brian	Roepke	broepke@gmail.com	5	13	True	+14155479222	1831699b-e255-45ff-8671-b5c840922735	$2b$12$eqnt856tWE.IyIUb5n3kNObd5TV8l69qo5CSJ4iODYyA7W5VCmusK
*/


In [5]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
  
database_chain = create_sql_query_chain(llm,db)

In [6]:
connection_parameters = {
            "account": snowflake_account,
            "user": username,
            "password": password,
            "role": role,
            "warehouse": warehouse,
            "database": database,
            "schema": schema

        }

session = Session.builder.configs(connection_parameters).create()

In [7]:
prompt = "Show me how many players we have in the game"
prompt = "Show me the name of the player that picked Joe Biden in 2024"
prompt = "Show me a list of all the picks by Brian Roepke for 2024"

sql_query = database_chain.invoke({"question": prompt})

#we can visualize what sql query is generated by the LLM
print(sql_query)

session.sql(sql_query).show()

SELECT name, year, wiki_page
FROM picks
WHERE picked_by = '1831699b-e255-45ff-8671-b5c840922735' AND year = 2024
LIMIT 5
--------------------------------------------------
|"NAME"             |"YEAR"  |"WIKI_PAGE"        |
--------------------------------------------------
|Yahya Sinwar       |2024    |Yahya_Sinwar       |
|Gerhard Schroeder  |2024    |Gerhard_Schroeder  |
|Dick Cheney        |2024    |Dick_Cheney        |
|Gene Shalit        |2024    |Gene_Shalit        |
|Warren Buffet      |2024    |Warren_Buffet      |
--------------------------------------------------

